[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openvinotoolkit/openvino_tensorflow/blob/master/examples/notebooks/Colab_OpenVINO_TensorFlow_object_detection_example.ipynb)



#**Object Detection with OpenVINO™ integration with TensorFlow**:

OpenVINO™ integration with TensorFlow is designed for TensorFlow developers who want to get started with OpenVINO™ in their inferencing applications. This product effectively delivers OpenVINO™ inline optimizations which enhance inferencing performance with minimal code modifications. OpenVINO™ integration with TensorFlow accelerates inference across many AI models on a variety of Intel® silicon such as: 
*   Intel® CPUs
*   Intel® integrated GPUs
*   Intel® Movidius™ Vision Processing Units - referred to as VPU
*   Intel® Vision Accelerator Design with 8 Intel Movidius™ MyriadX VPUs - referred to as VAD-M or HDDL

**Overview**

The following code demonstrates acceleration of YOLOv4 using OpenVINO™ integration with TensorFlow. We compare the performance of YOLOv4 with and without OpenVINO™ integration with TensorFlow. This model is a real time object detection algorithm that identifies objects in images & videos. It detects these objects by using features learned by a deep convolutional neural network.

In [ ]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

!pip3 -q install --upgrade pip
!pip3 -q install pillow
!pip3 -q install keras_applications

In [ ]:
!ldd --version

## INSTALL OpenVINO™ integration with TensorFlow

In [ ]:
# Install stock TensorFlow
!pip3 -q install tensorflow==2.7.0

# Install OpenVINO™ integration with TensorFlow
!pip3 -q install openvino-tensorflow

In [ ]:
# Download OpenVINO TensorFlow repository
!git clone --quiet https://github.com/openvinotoolkit/openvino_tensorflow
%cd openvino_tensorflow
!git submodule init
!git submodule update --recursive


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import tensorflow as tf
import openvino_tensorflow as ovtf
from PIL import Image
import cv2
import matplotlib.pyplot as plt

import sys
sys.path.append(os.getcwd() + '/examples')
from common.utils import get_input_mode, get_colors, draw_boxes, get_anchors, rename_file
from common.pre_process import preprocess_image_yolov3 as preprocess_image
from common.post_process import yolo3_postprocess_np

# Let's get the model





In [ ]:
# Download and Convert the YoloV4 model
%cd examples/
!apt install -qq virtualenv python3-venv
!chmod +x convert_yolov4.sh && bash convert_yolov4.sh &>/dev/null
%cd ../

In [ ]:
def load_coco_names(file_name):
    """Parses the label file with only class names,
      and returns a dictionary mapping the class IDs to class names.
    """
    names = {}
    with open(file_name) as f:
        for id, name in enumerate(f):
            names[id] = name
    return names

In [ ]:
def load_labels(label_file):
    """Parses the label file, assuming that labels are separated with a newline
       in the file and returns the list of labels.
    """  
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [ ]:
def infer_openvino_tensorflow(model_file, image_file , input_height, input_width, label_file, anchor_file, conf_threshold, iou_threshold):
    """Takes the tensorflow model and all other input parameters as arguments. 
       Runs inference with the object detection model and prints the predictions.
    """
    print("CREATE MODEL - BEGIN")

    # Load model and process input image
    model =     model = tf.saved_model.load(model_file)
    print("CREATE MODEL - END")

    if label_file:
        classes = load_coco_names(label_file)
        labels = load_labels(label_file)
        colors = get_colors(labels)

    if anchor_file:
        anchors = get_anchors(anchor_file)

    print("PREDICTION - BEGIN")
    
    #Preprocess Image
    image = Image.open(image_file)
    img = np.asarray(image)
    image_width, image_height = image.size
    img_resized = tf.convert_to_tensor(preprocess_image(image, (input_height, input_width)))

    # Warmup
    detected_boxes = model(img_resized)
    # Run
    import time
    start = time.time()
    detected_boxes = model(img_resized)
    elapsed = time.time() - start
    print('Inference time in ms: %f' % (elapsed * 1000))
    print("PREDICTION - END")  
    
    image_shape = tuple((image_height, image_width))  
    # apply non max suppresion, draw boxes and save updated image
    out_boxes, out_classes, out_scores = yolo3_postprocess_np(
            detected_boxes,
            image_shape,
            anchors,
            len(labels), (input_height, input_width),
            max_boxes=10,
            confidence=conf_threshold,
            iou_threshold=iou_threshold,
            elim_grid_sense=True)
    img_bbox = draw_boxes(img, out_boxes, out_classes, out_scores,
                        labels, colors)
    cv2.imwrite("detections.jpg", img_bbox)
    if output_dir:
        image.save(os.path.join(output_dir, "detections.jpg"))
    else:
        image.save("detections.jpg")
    plt.imshow(img)

# Now lets infer

*   Set all the parameters needed for inference
*   Enable OpenVINO™ integration with TensorFlow, and set Backend in just a few simple lines of code to boost performace
*   Infer the input image 
*   Output the predicted bounding box on the image, and the inference time with OpenVINO™ integration with TensorFlow enabled



In [ ]:
   input_file = "examples/data/grace_hopper.jpg"
   model_file = "examples/data/yolo_v4"
   label_file = "examples/data/coco.names"
   anchor_file = "examples/data/yolov4_anchors.txt"
   input_height = 416
   input_width = 416
   backend_name = "CPU"
   output_dir = "."
   conf_threshold = 0.6
   iou_threshold = 0.5

   #Print list of available backends
   print('Available Backends:')
   backends_list = ovtf.list_backends()
   for backend in backends_list:
       print(backend)
   ovtf.set_backend(backend_name)

   print("OpenVINO TensorFlow is enabled")
   infer_openvino_tensorflow(model_file, input_file, input_height, input_width, label_file, anchor_file, conf_threshold, iou_threshold )

*   Disable OpenVINO™ integration with TensorFlow to gauge the achieved performance boost
*   Infer the input image again
*   Output the predicted bounding box on the image, and the inference time with OpenVINO™ integration with TensorFlow disabled


In [ ]:
#Disable
ovtf.disable()
print("")
print("")
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, input_file, input_height, input_width, label_file, anchor_file, conf_threshold, iou_threshold )
ovtf.enable()